<a href="https://colab.research.google.com/github/giovit16/MIS2800/blob/main/IC_Feb04_IC_STUDENT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IC (Feb 04) — Text Data in SQL (Student Version)

This in-class exercise practices the ideas from the Feb 04 slides:
- Text / character data and why it’s tricky
- `GROUP BY` + `COUNT()`
- `ORDER BY` (frequency vs alphabetical)
- Case sensitivity and `LOWER()` / `UPPER()`
- Spaces and `TRIM()`
- `LIKE` searches (and why they can be surprising)

**Instructions:** Run cells top → bottom. Write SQL inside the `query` blocks.


## 0) Setup (Run)
Creates a SQLite database `text_issues.db` and a table `product` with messy text.

In [1]:
!pip install --upgrade pandas ipython-sql prettytable==3.10.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 71.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 63.3 MB/s eta 0:00:00
  Attempting uninstall: prettytable
    Found existing installation: prettytable 3.17.0
    Uninstalling prettytable-3.17.0:
      Successfully uninstalled prettytable-3.17.0
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 3.0.0 which is incompatible.
gradio 5.50.0 requires pandas<3.0,>=1.0, but you have pandas 3.0.0 which is incompatible.
bqplot 0.12.45 requires pandas<3.0.0,>=1.0.0, but you have pandas 3.0.0 which is

In [2]:
# Install the SQL extension (not in sources, but required for magics)
!pip install ipython-sql
print("ipython sql installed")
# lines of code will run below for the installation

ipython sql installed


In [3]:
# Load the SQL magic extension
%load_ext sql

In [4]:
import sqlite3, pandas as pd

In [5]:
# This creates a brand-new, empty database file named 'text_issues.db'
conn = sqlite3.connect('text_issues.db')
print("database created")

database created


In [6]:
%sql sqlite:///text_issues.db

In [7]:
# create a table called inventory_table
%%sql
CREATE TABLE inventory_table (
  id INTEGER PRIMARY KEY,
  category TEXT,
  note TEXT
) STRICT;

 * sqlite:///text_issues.db
Done.


[]

In [8]:
# complet the table creation
%%sql
INSERT INTO inventory_table (id, category, note)
VALUES
  (1,  "Apple",   "fresh"),
  (2,  "apple",   "fresh"),
  (3,  "APPLE",   "fresh"),
  (4,  "Banana",  "ripe"),
  (5,  "banana",  "ripe"),
  (6,  "banana ", "ripe"),
  (7,  " banana", "ripe"),
  (8,  "to-do",   "punctuation"),
  (9,  "to–do",   "punctuation"),
  (10, "",        "empty string"),
  (11, 'None',      "NULL value");

 * sqlite:///text_issues.db
11 rows affected.


[]

In [9]:
%%sql
SELECT * FROM inventory_table;

 * sqlite:///text_issues.db
Done.


id,category,note
1,Apple,fresh
2,apple,fresh
3,APPLE,fresh
4,Banana,ripe
5,banana,ripe
6,banana,ripe
7,banana,ripe
8,to-do,punctuation
9,to–do,punctuation
10,,empty string


In [11]:
%%sql
SELECT * FROM inventory_table;

 * sqlite:///text_issues.db
Done.


id,category,note
1,Apple,fresh
2,apple,fresh
3,APPLE,fresh
4,Banana,ripe
5,banana,ripe
6,banana,ripe
7,banana,ripe
8,to-do,punctuation
9,to–do,punctuation
10,,empty string


## Part A — Quick Concept Check (We do it together)
1. Which values in `category` refer to the *same* category but look different?
2. What is the difference between an **empty string** (`''`) and **NULL**?
3. Why might punctuation cause matching problems?


#1. apple vs APPLE, 'APPLE' vs 'aPPLE'

#2. empty string is a space, null is no value

#3. the machine will take different punctuation as different values

## Part B1 — Grouping and counting (raw text)
Which product categories have the most products, and how many products are in each category (sorted from most to least)

In [13]:
%%sql
-- Group by category and count.
SELECT category, COUNT(category) as count_number
FROM inventory_table
GROUP BY category
ORDER BY COUNT(category) DESC;




 * sqlite:///text_issues.db
Done.


category,count_number
to–do,1
to-do,1
banana,1
banana,1
apple,1
None,1
Banana,1
Apple,1
APPLE,1
banana,1


## Part B2 — Fix grouping with LOWER() - individual one

In [17]:
%%sql
-- Make category case-insensitive: group by LOWER(category) and count (ignore NULLs)
SELECT LOWER(category), COUNT(category)
FROM inventory_table
GROUP BY LOWER (category)
ORDER BY COUNT(category) DESC;


 * sqlite:///text_issues.db
Done.


LOWER(category),COUNT(category)
apple,3
banana,2
to–do,1
to-do,1
none,1
banana,1
banana,1
,1


## Part B3 — Fix spaces with TRIM() + LOWER()

In [32]:
%%sql

-- Clean spaces and case: group by LOWER(TRIM(category)) and count (ignore NULLs).
SELECT LOWER(TRIM(category)), COUNT(category)
FROM inventory_table
GROUP BY LOWER(trim(category))
ORDER BY COUNT(category) DESC;


 * sqlite:///text_issues.db
Done.


LOWER(TRIM(category)),COUNT(category)
banana,4
apple,3
to–do,1
to-do,1
none,1
,1


## Part C1 — Order by category (alphabetical)

In [30]:
%%sql
-- Show unique cleaned categories in alphabetical order.
SELECT LOWER(TRIM(category)) AS cleaned_category, COUNT(*)
FROM inventory_table
GROUP BY cleaned_category
ORDER BY cleaned_category ASC;





 * sqlite:///text_issues.db
Done.


cleaned_category,COUNT(*)
,1
apple,3
banana,4
none,1
to-do,1
to–do,1


## Part D1 — Case matters

In [26]:
%%sql
-- Return rows where category = 'apple' (exact match).
SELECT category, COUNT(category)
FROM inventory_table
ORDER BY COUNT (category) ASC;


 * sqlite:///text_issues.db
Done.


category,COUNT(category)
Apple,11


## Part D2 — Case-insensitive match

In [28]:
%%sql
-- Return rows that are apple regardless of case/spaces using LOWER(TRIM(category)) = 'apple'.
SELECT *
FROM inventory_table
WHERE LOWER(TRIM(category)) = 'apple';


 * sqlite:///text_issues.db
Done.


id,category,note
1,Apple,fresh
2,apple,fresh
3,APPLE,fresh


## Part D3 — Empty strings aren’t NULL

In [34]:
%%sql
-- Show rows where category is an empty string '' (NOT NULL).
SELECT *
FROM inventory_table
WHERE category = '';

 * sqlite:///text_issues.db
Done.


id,category,note
10,,empty string


## Part D4 — Find NULL categories

In [36]:
%%sql
-- Show rows where category IS NULL.
SELECT *
FROM inventory_table
WHERE category IS NULL;




 * sqlite:///text_issues.db
Done.


id,category,note


# Task
Implement a SQL query in cell `049ac8b6` that selects all rows from `inventory_table` where the `category` column, after being converted to lowercase and trimmed of leading/trailing spaces, is equal to 'apple'. Verify that only 'apple' categories (case and space insensitive) are returned.

## Implement Case-Insensitive Match

### Subtask:
Add a SQL query to cell `049ac8b6` that selects all rows from `inventory_table` where the `category` column, after being converted to lowercase and trimmed of leading/trailing spaces, is equal to 'apple'.


**Reasoning**:
The subtask requires writing a SQL query in cell `049ac8b6` to select rows where the `category` is 'apple' (case-insensitive and space-insensitive). I will add the SQL query to the specified cell.



## Final Task

### Subtask:
Verify the output of the query to ensure only 'apple' categories (case and space insensitive) are returned.


## Summary:

### Data Analysis Key Findings

*   The SQL query successfully filtered the `inventory_table` to include only rows where the `category` column, after being converted to lowercase and trimmed of leading/trailing spaces, matched 'apple'.
*   The `LOWER(TRIM(category)) = 'apple'` condition effectively handled case-insensitivity and leading/trailing spaces.
*   The query's output `[(1, 'Apple', 'fresh'), (2, 'apple', 'fresh'), (3, 'APPLE', 'fresh')]` confirmed that categories like 'Apple', 'apple', and 'APPLE' were all correctly identified and returned.

### Insights or Next Steps

*   The use of `LOWER(TRIM(column_name))` is an effective and robust method for performing case-insensitive and space-insensitive string comparisons in SQL queries.
*   This technique is valuable for standardizing data matching criteria, ensuring consistent results even when dealing with varied user inputs or inconsistent data entry.


# Task
Modify cell `ae34c82b` to display unique cleaned categories (using `LOWER(TRIM(category))`) and their counts, ordered alphabetically by the cleaned category, and verify the output.

## Implement Alphabetical Order for Cleaned Categories

### Subtask:
Modify cell `ae34c82b` to include a SQL query that shows unique cleaned categories (using `LOWER(TRIM(category))`) in alphabetical order, along with their counts.


**Reasoning**:
The subtask requires modifying cell `ae34c82b` to display unique cleaned categories in alphabetical order with their counts. I will provide the SQL query to achieve this by using `LOWER(TRIM(category))` for both selection, grouping, and ordering.



The SQL query in cell `ae34c82b` has been modified and executed. The output is:

```
[('', 1), ('apple', 3), ('banana', 4), ('none', 1), ('to-do', 1), ('to–do', 1)]
```

**Verification:**
*   **Unique Cleaned Categories**: The output clearly shows unique categories after `LOWER(TRIM(category))` is applied (e.g., 'apple', 'banana', 'to-do', 'to–do', '', 'none').
*   **Counts**: Each unique cleaned category is associated with its correct count (`apple`: 3, `banana`: 4, etc.).
*   **Alphabetical Order**: The categories are ordered alphabetically: `''`, `apple`, `banana`, `none`, `to-do`, `to–do`. This confirms the `ORDER BY cleaned_category ASC` clause worked as expected.

Thus, the output successfully meets all requirements of the subtask.

## Final Task

### Subtask:
Verify the output to ensure that categories are cleaned (case and space insensitive), grouped correctly, and ordered alphabetically.


## Summary:

### Q&A
Yes, the output verifies that categories are cleaned (case and space insensitive), grouped correctly, and ordered alphabetically. The `LOWER(TRIM(category))` function successfully cleaned categories, and the `GROUP BY` and `ORDER BY` clauses ensured correct grouping and alphabetical ordering.

### Data Analysis Key Findings
*   The SQL query successfully identified unique cleaned categories, which included `''` (empty string), `apple`, `banana`, `none`, `to-do`, and `to–do`.
*   Each unique cleaned category was associated with its correct count; for example, 'apple' appeared 3 times, and 'banana' appeared 4 times.
*   The results were correctly ordered alphabetically by the `cleaned_category`, starting with `''`, followed by `apple`, `banana`, `none`, `to-do`, and `to–do`.
*   While `LOWER(TRIM(category))` handled case and leading/trailing spaces, variations like 'to-do' and 'to–do' (distinguished by different dash characters) were treated as separate categories, indicating the cleaning was specific to those functions.

### Insights or Next Steps
*   Further data cleaning might be necessary to normalize similar categories such as 'to-do' and 'to–do' that differ only by special characters. This could involve using additional string manipulation functions to standardize character types.


# Task
Modify the SQL query in cell `dd65473d` by replacing `COUNT (ignore nulls)` with `COUNT(category)` to resolve the syntax error, then verify that the output correctly displays categories cleaned using `LOWER(TRIM(category))`, grouped, and counted in descending order without any errors.

## Correct SQL Query in Part B3

### Subtask:
Modify the SQL query in cell `dd65473d` to use `COUNT(category)` instead of `COUNT (ignore nulls)` to fix the syntax error. The query should group by `LOWER(TRIM(category))` and order by the count in descending order.


**Reasoning**:
The subtask explicitly asks to modify the SQL query in cell `dd65473d` to correct a syntax error by changing `COUNT (ignore nulls)` to `COUNT(category)`, while retaining the grouping and ordering clauses. I will update the code in cell `dd65473d` accordingly.



## Final Task

### Subtask:
Verify the output of cell dd65473d to ensure categories are correctly cleaned, grouped, and counted, and that the query runs without syntax errors.


## Summary:

### Q&A
The SQL query in cell `dd65473d` now runs without syntax errors after changing `COUNT (ignore nulls)` to `COUNT(category)`. The output correctly displays categories cleaned using `LOWER(TRIM(category))`, grouped, and counted in descending order.

### Data Analysis Key Findings
*   The original SQL query had a syntax error with `COUNT (ignore nulls)`, which was resolved by replacing it with `COUNT(category)`.
*   The `LOWER(TRIM(category))` function successfully cleaned and standardized category names, consolidating variations such as 'Banana', 'banana', and ' banana' into a single 'banana' entry.
*   The query accurately grouped and counted items for each cleaned category. For instance, 'banana' appeared 4 times, 'apple' 3 times, and 'to–do', 'to-do', 'none', and '' each appeared once.
*   The results were correctly ordered in descending order based on the count of items per category, with the most frequent categories listed first.

### Insights or Next Steps
*   This cleaning and counting approach effectively identifies the most common inventory categories, which can inform inventory management and purchasing decisions.
*   Consider standardizing variations like 'to–do' and 'to-do' to a single entry in the source data or via a more comprehensive cleaning step to further improve data consistency.


# Task
Implement a SQL query in cell `1acf51d6` that selects all rows from `inventory_table` where the `category` column is an empty string (`''`), then verify the output to ensure that only rows with an empty string in the `category` column are returned, confirming that empty strings are not treated as NULL.

## Implement Query for Empty Strings

### Subtask:
Add a SQL query to cell `1acf51d6` that selects all rows from `inventory_table` where the `category` column is an empty string (`''`).


**Reasoning**:
The subtask requires adding a SQL query to cell `1acf51d6` that selects rows where the `category` is an empty string. I will add the SQL query to the specified cell.



## Final Task

### Subtask:
Verify the output to ensure that only rows with an empty string in the `category` column are returned, confirming that empty strings are not treated as NULL.


## Summary:

### Data Analysis Key Findings
*   The SQL query successfully retrieved rows from `inventory_table` where the `category` column was an empty string (`''`).
*   The query returned one specific row: `(10, '', 'empty string')`.
*   This outcome confirms that the database correctly interprets empty strings as distinct values, rather than treating them as `NULL`.

### Insights or Next Steps
*   The system accurately differentiates between empty strings and `NULL` values, ensuring precise data filtering based on string content.


# Task
Implement a SQL query in cell `04a68c98` that selects all rows from `inventory_table` where the `category` column IS NULL.

## Find NULL Categories

### Subtask:
Add a SQL query to cell `04a68c98` that selects all rows from `inventory_table` where the `category` column IS NULL.


**Reasoning**:
The subtask requires adding a SQL query to cell `04a68c98` that selects rows where the `category` is NULL. I will add the SQL query to the specified cell.



## Final Task

### Subtask:
Verify the output of the query to ensure that only rows with a NULL value in the `category` column are returned.

## Final Task

### Subtask:
Verify the output to ensure that only rows with a NULL value in the `category` column are returned, confirming the distinction between NULL and empty strings.


## Summary:

### Data Analysis Key Findings

*   The SQL query executed to find `NULL` values in the `category` column returned an empty result set.
*   This indicates that there are no actual SQL `NULL` values in the `category` column of the `inventory_table`.
*   The `IS NULL` operator specifically targets SQL `NULL`s and does not identify other representations of "no category," such as empty strings or text like 'None'.

### Insights or Next Steps

*   To identify items without an assigned category, investigate other potential representations, such as empty strings (''), specific placeholder strings ('None', 'N/A'), or default values, and adjust the SQL query accordingly (e.g., `WHERE category IS NULL OR category = '' OR category = 'None'`).
